In [1]:
import sys
sys.path.append("../")

In [3]:
import pandas as pd
import datetime as dt
from technicals.indicators import MACD
from simulation.guru_tester import GuruTester

In [4]:
BUY = 1
SELL = -1
NONE = 0

def apply_signal(row):
    if row.direction == BUY and row.mid_l > row.EMA_100:
        return BUY
    if row.direction == SELL and row.mid_h < row.EMA_100:
        return SELL
    return NONE        

In [5]:
pair = "EUR_USD"
df_an = pd.read_pickle(f"../data/{pair}_H1.pkl")
df_m5 = pd.read_pickle(f"../data/{pair}_M5.pkl")
df_an.reset_index(drop=True, inplace=True)
df_m5.reset_index(drop=True, inplace=True)

In [6]:
df_an = MACD(df_an)
df_an['EMA_100'] = df_an.mid_c.ewm(span=100, min_periods=100).mean()
df_an.dropna(inplace=True)
df_an.reset_index(drop=True, inplace=True)

In [10]:

def apply_cross(row):
    #buy cross
    if row.macd_delta > 0  and row.macd_delta_prev < 0:
        return BUY
    #sell cross
    if row.macd_delta < 0  and row.macd_delta_prev > 0:
        return SELL
    return NONE

In [11]:
df_an['macd_delta'] = df_an.MACD - df_an.SIGNAL
df_an['macd_delta_prev'] = df_an.macd_delta.shift(1)
df_an['direction'] = df_an.apply(apply_cross, axis=1)

In [12]:
df_an.direction.value_counts()

direction
 0    34172
 1     1473
-1     1472
Name: count, dtype: int64

In [13]:
df_an.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,...,ask_h,ask_l,ask_c,MACD,SIGNAL,HIST,EMA_100,macd_delta,macd_delta_prev,direction
0,2017-01-13 01:00:00+00:00,1004,1.06098,1.06198,1.06084,1.06164,1.06091,1.06190,1.06076,1.06157,...,1.06206,1.06091,1.06171,0.000518,0.001241,-0.000723,1.059109,-0.000723,NaN,0
1,2017-01-13 02:00:00+00:00,902,1.06162,1.06190,1.06038,1.06054,1.06154,1.06183,1.06031,1.06047,...,1.06198,1.06044,1.06061,0.000329,0.001059,-0.000730,1.059142,-0.000730,-0.000723,0
2,2017-01-13 03:00:00+00:00,639,1.06057,1.06122,1.06042,1.06080,1.06050,1.06114,1.06034,1.06073,...,1.06129,1.06049,1.06087,0.000197,0.000886,-0.000689,1.059179,-0.000689,-0.000730,0
3,2017-01-13 04:00:00+00:00,382,1.06077,1.06175,1.06056,1.06175,1.06070,1.06168,1.06048,1.06168,...,1.06182,1.06063,1.06182,0.000167,0.000743,-0.000575,1.059238,-0.000575,-0.000689,0
4,2017-01-13 05:00:00+00:00,363,1.06173,1.06173,1.06126,1.06170,1.06164,1.06166,1.06119,1.06164,...,1.06182,1.06132,1.06177,0.000138,0.000622,-0.000483,1.059294,-0.000483,-0.000575,0


In [14]:
def run_pair(pair, time_d=1):
    df_an = pd.read_pickle(f"../data/{pair}_H{time_d}.pkl")
    df_m5 = pd.read_pickle(f"../data/{pair}_M5.pkl")
    df_an.reset_index(drop=True, inplace=True)
    df_m5.reset_index(drop=True, inplace=True)
    df_an = MACD(df_an) #MACD over RSI here
    df_an['EMA_100'] = df_an.mid_c.ewm(span=100, min_periods=100).mean()

    #direction created
    df_an['macd_delta'] = df_an.MACD - df_an.SIGNAL
    df_an['macd_delta_prev'] = df_an.macd_delta.shift(1)
    df_an['direction'] = df_an.apply(apply_cross, axis=1)
    
    our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
            'bid_o', 'bid_h', 'bid_l', 'bid_c', 
            'ask_o', 'ask_h', 'ask_l', 'ask_c',
             'direction', 'EMA_100' ]
    df_slim = df_an[our_cols].copy()
    df_slim.dropna(inplace=True)
    
    df_slim.reset_index(drop=True, inplace=True)
    gt = GuruTester(
        df_slim,
        apply_signal,
        df_m5,
        use_spread=True,
        time_d=time_d
    )
    
    gt.run_test()
    return gt.df_results
    

In [15]:
res = []
for p in ["AUD_NZD", "EUR_USD", "GBP_JPY", "USD_CAD" ]:
    res.append(dict(pair=p,res=run_pair(p, time_d=1)))

In [16]:
for r in res:
    print(r['pair'], r['res'].result.sum())

AUD_NZD -524.0
EUR_USD -51.0
GBP_JPY -6.0
USD_CAD -118.5


In [17]:
res = []
for p in ["AUD_NZD", "EUR_USD", "GBP_JPY", "USD_CAD" ]:
    res.append(dict(pair=p,res=run_pair(p, time_d=4)))
for r in res:
    print(r['pair'], r['res'].result.sum())

AUD_NZD -65.0
EUR_USD 11.5
GBP_JPY 35.5
USD_CAD -23.0
